In [2]:
import modern_robotics as mr
import numpy as np

R = 0.0475 # (const) wheel radius
L = 0.235 # (const) distance to wheel in x_b direction
W = 0.15 # (const) distance to wheel in y_b direction


**NextState:**

Args:
- curr_state  
    Vector (12x1) containing the current angle of each joint
    - Chasis (0 - 2)
    - Arm Joints (3 - 7)
    - Wheel (8 - 11)
- velocities  
    Vector (9x1) containing joint and wheel velocities
    - Arm Joints (0 - 4)
    - Wheel Velocities (5 - 8)
- dt  
    (float) value of the timestep
- max_vel  
    (float) maximum velocity for joint and wheel movements

In [3]:
def NextState(curr_state: np.ndarray, velocities: np.ndarray, dt: float, max_vel: float) -> np.ndarray:
    next_state = np.zeros(12)

    velocities[np.where(velocities > max_vel)] = max_vel
    velocities[np.where(velocities < -max_vel)] = -max_vel

    # determine the new joint angles
    next_state[3:8] = curr_state[3:8] + dt * velocities[4:]

    # determine the new wheel angles
    next_state[8:] = curr_state[8:] + dt * velocities[:4]

    # calculate the change in wheel angles
    d_wheel_angles = next_state[8:] - curr_state[8:]

    # eqn 13.10 (pg. 541)
    H_0 = (1/R) *  np.array([[-L-W, 1, -1], [L+W, 1, 1], [L+W, 1, -1], [-L-W, 1, 1]])

    # body twist is a 3x1 vector
    # eqn 13.33 (pg. 569)
    #   w_bz
    #   v_bx
    #   v_by
    body_twist = np.matmul(np.linalg.pinv(H_0), d_wheel_angles)

    # eqn 13.35 (pg. 570)
    dq_b = np.zeros(3)
    if body_twist[0] == 0.0:
        dq_b = body_twist
    else:
        dq_b = np.array([
            body_twist[0],
            ( body_twist[1] * np.sin(body_twist[0]) + body_twist[2] * (np.cos(body_twist[0]) - 1) ) / body_twist[0],
            ( body_twist[2] * np.sin(body_twist[0]) + body_twist[1] * (1 - np.cos(body_twist[0])) ) / body_twist[0]
        ])
    
    # eqn 13.36 (pg 570)
    dq = np.matmul(np.array([
        [1, 0, 0],
        [0, np.cos(curr_state[0]), -np.sin(curr_state[0])],
        [0, np.sin(curr_state[0]), np.cos(curr_state[0])],
    ]), dq_b)

    next_state[:3] = curr_state[:3] + dq

    return next_state    

Testing the Code:

In [4]:
import csv

dt = 0.01
N = 100

curr_state = np.zeros(12)
velocities = np.ones(9)
velocities[4:] *= 0
velocities[0] *= -10
velocities[1] *= 10
velocities[2] *= 10
velocities[3] *= -10

with open("test.csv", "w+") as file:
    writer = csv.writer(file)
    write_data = np.zeros(13)

    for _ in range(N):
        write_data[:12] = curr_state
        writer.writerow(write_data.round(4))
        curr_state = NextState(curr_state, velocities, dt, 20)

# Video Link
[link](https://drive.google.com/file/d/1wd70hRHvS_79FOPj-ytq6ssR58TkKqdT/view?usp=sharing)